In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split

In [2]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

In [ ]:
import tensorflow as tf
# print(tf.__version__)
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
from create_graphs import Graph 

In [ ]:
N = 8
max_qtd = 782
epochs = 100

In [ ]:
X, y = [], []
for i in range(max_qtd):
    g = Graph(N)
    X.append(g.get_flatten_matrix())
    y.append(g.get_chrome_number())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [ ]:
def binary2int(key):
    if not isinstance(key, str):
        key = str(key)

    return int(key, 2)


In [ ]:
def bit_length(number):
    if not isinstance(number, int):
        number = int(number)

    return (number).bit_length()


In [ ]:
def max_value(dic):
    max_key = max(dic, key=dic.get)

    max_value = dic[max_key]

    return max_key, max_value


In [ ]:
def feature_map(inputs, bit_length_response):

    qReg = QuantumRegister(len(inputs))
        
    cReg = ClassicalRegister(bit_length_response)
    qc = QuantumCircuit(qReg, cReg)
    
    for i, x in enumerate(inputs):
        qc.rx(x, i)
    
    qc.barrier()
    
    return qc, cReg


In [ ]:
def variational_circuit(qc, theta, inputs):
    
    for i in range(len(inputs) - 1):
        qc.cx(i+1, i)
    
    qc.cx(0, len(inputs)-1)
    qc.barrier()
    
    for i in range(len(inputs)):
        qc.ry(theta[i], i)

    qc.barrier()

    return qc


In [ ]:
def quantum_nn(inputs, theta, bit_length_response, shots, with_errors=False):
    inputs_np = inputs.numpy()[-1]
    theta_np = theta.numpy()

    qc, cReg = feature_map(inputs_np, bit_length_response)
    qc = variational_circuit(qc, theta_np, inputs_np)

    for i in range(bit_length_response):
        qc.measure(i, cReg[i])

    results = AerSimulator().run(qc, shots=shots, memory=True).result()
    if with_errors:
        
    counts = results.get_counts(qc)

    stats = {}
    for key, value in counts.items():
        stats[binary2int(key)] = value / shots

    max_key, max_v = max_value(stats)

    return max_key, max_v


In [ ]:
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self, bit_length_response, shots=1E4, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.bit_length_response = bit_length_response
        self.shots = shots

    def build(self, input_shape):
        super(QuantumLayer, self).build(input_shape)
        self.N = input_shape[-1]
        self.n_response = bit_length(self.bit_length_response) # self.bit_length_response
        self.theta = self.add_weight(name='theta', shape=(self.N,), initializer='random_normal', trainable=True)

    def call(self, inputs):
        if tf.executing_eagerly():
            output_key, output_value = quantum_nn(inputs, self.theta, self.bit_length_response, self.shots)
            # print(f"Key: {max_key}")

            outputs = np.zeros((tf.shape(inputs)[0], self.N))  # Modificação para retornar um tensor bidimensional
            if not output_key >= self.N:
                outputs[0, output_key] = output_value

            outputs = tf.constant(outputs, dtype=tf.float32)

            return outputs

        return inputs

    def get_config(self):
        config = super(QuantumLayer, self).get_config()
        config.update({
            'bit_length_response': self.bit_length_response,
            'shots': self.shots,
        })
        return config


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(N, activation=tf.nn.relu, name="Dense_1"),
    QuantumLayer(N, name="QuantumLayer"),
    tf.keras.layers.Dense(N+1, activation=tf.nn.softmax, name="Dense_2")
])
model.build()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

In [ ]:
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Acurácia de Treino')
plt.plot(range(1, len(test_accuracy) + 1), test_accuracy, label='Acurácia de Teste')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.title('Acurácia ao longo das Épocas')
plt.legend()
plt.show()

In [ ]:
exemplo_test = X_test

predict = model.predict(exemplo_test)

print(f"Valor real: {X_test[0]}\n\n Valor previsto: {y_test}")